In [32]:
import google.auth
import googleapiclient.discovery
from googleapiclient import discovery
import pandas as pd
from IPython.display import JSON
import requests
from config import youtube_api_key
requests.__version__
import json

In [33]:
channel_ids = ['UCDfbsXLRCyk_LIkrtaTmliQ', 
               #more channels here
              ]

In [34]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = discovery.build(
    api_service_name, api_version, developerKey=youtube_api_key)

request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=",".join(channel_ids)
)
response = request.execute()

print(response)

{'kind': 'youtube#channelListResponse', 'etag': 'pZW9Rs9P4FpXAxvdLzZ1kX5IzZ4', 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#channel', 'etag': 'Jv_uX_oyimRE-CvsJ2EqgBdjfGA', 'id': 'UCDfbsXLRCyk_LIkrtaTmliQ', 'snippet': {'title': 'PlanetaJuan', 'description': 'Larga vida a la abuela', 'customUrl': '@planetajuan', 'publishedAt': '2017-03-08T23:37:45Z', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AL5GRJXSQtfIvsusEgUE2E_jAKORPkqpTI7rmD9m2rSBxA=s88-c-k-c0x00ffffff-no-rj', 'width': 88, 'height': 88}, 'medium': {'url': 'https://yt3.ggpht.com/ytc/AL5GRJXSQtfIvsusEgUE2E_jAKORPkqpTI7rmD9m2rSBxA=s240-c-k-c0x00ffffff-no-rj', 'width': 240, 'height': 240}, 'high': {'url': 'https://yt3.ggpht.com/ytc/AL5GRJXSQtfIvsusEgUE2E_jAKORPkqpTI7rmD9m2rSBxA=s800-c-k-c0x00ffffff-no-rj', 'width': 800, 'height': 800}}, 'localized': {'title': 'PlanetaJuan', 'description': 'Larga vida a la abuela'}, 'country': 'US'}, 'contentDetails': {'relatedPlaylists': {'likes':

In [35]:
def get_channel_stats(youtube, channel_ids):
    all_data= []
    
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()
    
    # loop through items
    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos':item['statistics']['videoCount'],
                'playlistId':item['contentDetails']['relatedPlaylists']['uploads']
        }
        
        all_data.append(data)
    return(pd.DataFrame(all_data))

In [36]:
channel_stats = get_channel_stats(youtube, channel_ids)

In [37]:
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,PlanetaJuan,1480000,188418419,424,UUDfbsXLRCyk_LIkrtaTmliQ


In [38]:
playlist_id = "PL3-PXpEvVoG8zY7MFlb2PSfPzb6hNbv-A"

def get_video_ids(youtube, playlist_id):
    video_ids = []
    
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId="PL3-PXpEvVoG8zY7MFlb2PSfPzb6hNbv-A",
        maxResults = 50
    )
    response = request.execute()
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken') 
    while next_page_token is not None:
        request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                playlistId="PL3-PXpEvVoG8zY7MFlb2PSfPzb6hNbv-A",
                maxResults = 50,
                pageToken = next_page_token)
        response = request.execute()
        
        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])
        
        next_page_token = response.get('nextPageToken')
    
    return video_ids

In [39]:
video_ids = get_video_ids(youtube, playlist_id)

In [40]:
len(video_ids)

27

In [41]:
def get_video_details(youtube, video_ids):
    all_video_info = []
    for i in range(0, len(video_ids), 50):    
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id="Ks-_Mh1QhMc"
        )
        response = request.execute()


        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails':['duration','definition','caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']
    
        for k in stats_to_keep.keys():
            for v in stats_to_keep[k]:
                try: 
                    video_info[v] = video[k][v]
                except: 
                    video_info[v] = None

        all_video_info.append(video_info)
        
    return pd.DataFrame(all_video_info)

In [42]:
video_df = get_video_details(youtube, video_ids)
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,Ks-_Mh1QhMc,TED,Your body language may shape who you are | Amy...,"Body language affects how others see us, but i...","[Amy Cuddy, TED, TEDTalk, TEDTalks, TED Talk, ...",2012-10-01T15:27:35Z,22874968,382238,None,9480,PT21M3S,hd,true


In [43]:
video_df.tail()

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,Ks-_Mh1QhMc,TED,Your body language may shape who you are | Amy...,"Body language affects how others see us, but i...","[Amy Cuddy, TED, TEDTalk, TEDTalks, TED Talk, ...",2012-10-01T15:27:35Z,22874968,382238,None,9480,PT21M3S,hd,true


In [44]:
def get_comments_in_videos(youtube, video_ids):
    comments_data = []

    for video_id in response_dict:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            textFormat="plainText",
            maxResults=100
        )
        response = request.execute()
        response_dict = json.loads(response)

        comments_in_video = [
            comment['snippet']['topLevelComment']['snippet']['textOriginal'] 
            for comment in response_dict['items']
        ]
        comments_in_video_info = {'video_id': video_id, 'comments': comments_in_video}

        comments_data.append(comments_in_video_insfo)

    return pd.DataFrame(comments_data)